<a href="https://colab.research.google.com/github/smomin1/Calgary-Traffic-Risk-Forecaster/blob/main/Calgary_Traffic_Risk_Forecaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

In [3]:
#Imports

import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import wkt
import itertools
import warnings
import re


warnings.filterwarnings('ignore')

WGS84 = "EPSG:4326"
PROJECTED_CRS = 26911  # Calgary UTM Zone 11N (accurate area + faster spatial join)


# Data Integration, Cleaning, and Feature Engineering

In [6]:
# Load Raw Data

df_incidents = pd.read_csv("Traffic_Incidents_20260216.csv")
df_climate = pd.read_csv("calgary_climate_daily_2016_2026.csv")
df_population = pd.read_csv("Historical_Community_Populations_20260216.csv")
df_boundaries = pd.read_csv("Community_District_Boundaries_20260216.csv")


In [7]:
# Climate Processing

numeric_cols = [
    "Mean Temp (°C)",
    "Total Snow (cm)",
    "Total Precip (mm)",
    "Spd of Max Gust (km/h)"
]

for col in numeric_cols:
    df_climate[col] = pd.to_numeric(df_climate[col], errors="coerce")

df_climate["Total Snow (cm)"] = df_climate["Total Snow (cm)"].fillna(0)
df_climate["Total Precip (mm)"] = df_climate["Total Precip (mm)"].fillna(0)

df_climate["Date/Time"] = pd.to_datetime(df_climate["Date/Time"])
df_climate["year_month"] = df_climate["Date/Time"].dt.to_period("M")

monthly_climate = (
    df_climate
    .groupby("year_month")
    .agg(
        avg_mean_temp=("Mean Temp (°C)", "mean"),
        total_snow_cm=("Total Snow (cm)", "sum"),
        total_precip_mm=("Total Precip (mm)", "sum"),
        snow_days=("Total Snow (cm)", lambda x: (x > 0).sum()),
        high_wind_days=("Spd of Max Gust (km/h)", lambda x: (x > 50).sum())
    )
    .reset_index()
)


In [8]:
# Spatial Boundaries

df_boundaries["geometry"] = df_boundaries["MULTIPOLYGON"].apply(wkt.loads)

gdf_boundaries = gpd.GeoDataFrame(
    df_boundaries,
    geometry="geometry",
    crs=WGS84
)

# Project for accurate area
gdf_boundaries = gdf_boundaries.to_crs(epsg=PROJECTED_CRS)

gdf_boundaries["area_km2"] = gdf_boundaries.area / 1_000_000

neighborhood_info = gdf_boundaries[["COMM_CODE", "NAME", "area_km2"]]

print("Boundary processing complete.")


Boundary processing complete.


In [9]:
# Incident Processing

# Convert START and MODIFIED times
df_incidents["START_DT"] = pd.to_datetime(df_incidents["START_DT"], errors='coerce')
df_incidents["MODIFIED_DT"] = pd.to_datetime(df_incidents["MODIFIED_DT"], errors='coerce')

# Drop rows with missing start dates
df_incidents = df_incidents.dropna(subset=["START_DT"])

# Create year_month column
df_incidents["year_month"] = df_incidents["START_DT"].dt.to_period("M")

# # Calculate incident duration (hours)
# df_incidents["duration_hours"] = (df_incidents["MODIFIED_DT"] - df_incidents["START_DT"]).dt.total_seconds() / 3600
# df_incidents["duration_hours"] = df_incidents["duration_hours"].clip(lower=0).fillna(0)

# Severity scoring system
def severity_score(desc):
    """
    Classifies incidents based on description text.

    Severity scale:
        0 → normal traffic incident
        1 → multi-vehicle or two-vehicle incidents
        2 → incidents involving pedestrians or EMS on site
    Lane blockage and duration enhance severity.
    """
    desc = str(desc).lower()
    score = 0

    # High severity: pedestrian or EMS involvement
    if "pedestrian" in desc or "ems" in desc:
        score = 2

    # Medium severity: multi-vehicle or two-vehicle incidents
    elif "multi-vehicle" in desc or "two vehicle" in desc:
        score = 1

    # lane-blocking contribution
    if "blocking multiple lanes" in desc:
        score += 1
    elif re.search(r"blocking (left|right|middle|shoulder)", desc):
        score += 0.5

    return score

# Apply severity scoring
df_incidents["severity_score"] = df_incidents["DESCRIPTION"].apply(severity_score)

# #  Take incident duration into consideration for severity
# def enhance_with_duration(row):
#     score = row["severity_score"]

#     if row["duration_hours"] >= 2:       # Incident lasted 2+ hours
#         score += 1
#     elif row["duration_hours"] >= 1:     # Incident lasted 1–2 hours
#         score += 0.5

#     return score

# df_incidents["severity_score"] = df_incidents.apply(enhance_with_duration, axis=1)

# 6. Create binary "is_severe" flag

df_incidents["is_severe"] = (df_incidents["severity_score"] >= 1).astype(int)

print("Incident cleaning & enhanced severity scoring complete.")


Incident cleaning & enhanced severity scoring complete.


In [10]:
# Spatial Join

gdf_incidents = gpd.GeoDataFrame(
    df_incidents,
    geometry=gpd.points_from_xy(df_incidents.Longitude, df_incidents.Latitude),
    crs=WGS84
)

# Project incidents to match boundaries
gdf_incidents = gdf_incidents.to_crs(epsg=PROJECTED_CRS)

joined_incidents = gpd.sjoin(
    gdf_incidents,
    gdf_boundaries[["COMM_CODE", "geometry"]],
    how="inner",
    predicate="intersects"
)

incident_summary = (
    joined_incidents
    .groupby(["COMM_CODE", "year_month"])
    .agg(
        total_incidents=("id", "count"),
        severe_incidents=("severity_score", lambda x: (x > 0).sum()),
        avg_severity=("severity_score", "mean")
    )
    .reset_index()
)

print("Spatial join complete.")


Spatial join complete.


In [11]:
# Panel Construction


all_months = pd.period_range(
    df_incidents["year_month"].min(),
    df_incidents["year_month"].max(),
    freq="M"
)

all_comms = neighborhood_info["COMM_CODE"].unique()

base_panel = pd.DataFrame(
    list(itertools.product(all_comms, all_months)),
    columns=["COMM_CODE", "year_month"]
)

panel_df = (
    base_panel
    .merge(neighborhood_info, on="COMM_CODE", how="left")
    .merge(incident_summary, on=["COMM_CODE", "year_month"], how="left")
    .merge(monthly_climate, on="year_month", how="left")
)

panel_df.fillna(0, inplace=True)

print("Panel constructed.")


Panel constructed.


In [12]:
# Population Merge

df_population = df_population.copy()
df_population.columns = df_population.columns.str.strip().str.lower()

# Clean population column
df_population["population"] = (
    df_population["population"]
    .astype(str)
    .str.replace(",", "")
    .astype(float)
)

# Merge using COMM_CODE only
panel_df = panel_df.merge(
    df_population[["comm_code", "population"]],
    left_on="COMM_CODE",
    right_on="comm_code",
    how="left"
)

panel_df.drop(columns=["comm_code"], inplace=True)

# Remove zero or missing population areas
panel_df = panel_df[panel_df["population"] > 0]

print("Population merged.")


Population merged.


In [13]:
# Feature Engineering

panel_df["population_density"] = panel_df["population"] / panel_df["area_km2"]
panel_df["incidents_per_1000"] = (panel_df["total_incidents"] / panel_df["population"]) * 1000
panel_df["severe_per_1000"] = (panel_df["severe_incidents"] / panel_df["population"]) * 1000
panel_df["incidents_per_km2"] = panel_df["total_incidents"] / panel_df["area_km2"]

# Climate nonlinear features
panel_df["snow_squared"] = panel_df["total_snow_cm"] ** 2
panel_df["wind_snow_interaction"] = panel_df["total_snow_cm"] * panel_df["high_wind_days"]
panel_df["cold_days_flag"] = (panel_df["avg_mean_temp"] < -15).astype(int)

# Seasonality encoding
panel_df["month"] = panel_df["year_month"].dt.month
panel_df["sin_month"] = np.sin(2 * np.pi * panel_df["month"] / 12)
panel_df["cos_month"] = np.cos(2 * np.pi * panel_df["month"] / 12)

print("Advanced features created.")


Advanced features created.


In [14]:
# Lag & Rolling Features

panel_df = panel_df.sort_values(["COMM_CODE", "year_month"])

group = panel_df.groupby("COMM_CODE")

panel_df["lag_1"] = group["total_incidents"].shift(1)
panel_df["lag_3_avg"] = (
    group["total_incidents"]
    .rolling(3)
    .mean()
    .reset_index(level=0, drop=True)
)

panel_df["rolling_std_6"] = (
    group["total_incidents"]
    .rolling(6)
    .std()
    .reset_index(level=0, drop=True)
)

panel_df.fillna(0, inplace=True)

print("Lag & rolling features added.")


Lag & rolling features added.


In [15]:
# Target Definition

# Absolute threshold
# panel_df["high_risk"] = (panel_df["severe_per_1000"] > 0.4).astype(int)
panel_df["high_risk"] = panel_df.groupby("year_month")["severe_per_1000"] \
                                .transform(lambda x: x > x.quantile(0.75)).astype(int)


print("Target variable created.")


Target variable created.


In [16]:
# Final Dataset

panel_df.rename(columns={"NAME": "community_name"}, inplace=True)

panel_df = panel_df.sort_values(["COMM_CODE", "year_month"]).reset_index(drop=True)

print("feature table complete.")
panel_df.head()


feature table complete.


,COMM_CODE,year_month,community_name,area_km2,total_incidents,severe_incidents,avg_severity,avg_mean_temp,total_snow_cm,total_precip_mm,...,snow_squared,wind_snow_interaction,cold_days_flag,month,sin_month,cos_month,lag_1,lag_3_avg,rolling_std_6,high_risk
0,ABB,2016-12,ABBEYDALE,1.70414,0.0,0.0,0.0,-10.316667,26.1,22.4,...,681.21,208.8,0,12,-2.449294e-16,1.000000e+00,0.0,0.000000,0.0,0
1,ABB,2017-01,ABBEYDALE,1.70414,0.0,0.0,0.0,-6.766667,14.7,10.5,...,216.09,88.2,0,1,5.000000e-01,8.660254e-01,0.0,0.000000,0.0,0
2,ABB,2017-02,ABBEYDALE,1.70414,0.0,0.0,0.0,-7.081481,35.8,30.8,...,1281.64,143.2,0,2,8.660254e-01,5.000000e-01,0.0,0.000000,0.0,0
3,ABB,2017-03,ABBEYDALE,1.70414,1.0,0.0,0.0,-2.412903,16.7,16.2,...,278.89,116.9,0,3,1.000000e+00,6.123234e-17,0.0,0.333333,0.0,0
4,ABB,2017-04,ABBEYDALE,1.70414,0.0,0.0,0.0,4.800000,16.6,67.3,...,275.56,49.8,0,4,8.660254e-01,-5.000000e-01,1.0,0.333333,0.0,0


In [17]:
# panel_df.to_csv('df_panel.csv', index=False)
# from google.colab import files
# files.download('df_panel.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>